# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from collections import Counter
from pymystem3 import Mystem

In [2]:
df = pd.read_csv('/datasets/data.csv')

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Итак, в таблице 12 столбцов. Тип данных: int64, float64, object.
В названиях колонок не видны нарушения стиля.

1. Количество значений в столбцах `days_employed` и `total_income` различается. Значит, в данных есть пропущенные значения.
2. В столбце `days_employed` присутствуют отрицательные значения.
3. В столбце `education` встречаются неявные дубликаты.
4. Столбцы `total_income` и `days_employed` следует привести к целочисленным значениям.

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Отвечая на первый вопрос, столкнулся с неправильными данными в столбце `children` (количество детей: -1 и 20)

In [5]:
df['children'] = df['children'].replace(20, 2)
df['children'] = df['children'].replace(-1, 1)

## Шаг 2. Предобработка данных

### Обработка пропусков

In [6]:
# подсчёт пропусков
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенные значения, возможно, появились из-за отсутствия данных у банка, либо клиенты не захотели предоставить информацию об опыте работы и ежемесячных доходах, либо заемщики не имели опыта работы и соотвественно не получали зарплату. Также нельзя исключать, что пропуски в столбцах появились из-за неправильной записи данных.

Пропуски в столбцах `days_employed` и `total_income` можно заменить на медианные значения обоих столбцов (среднее значение не подойдет, так как в данных присутствуют аномальные значения). Предварительно необходимо привести все значения в столбце `days_employed` к модулю функцей `abs()`.

In [7]:
df['days_employed'] = df['days_employed'].abs()

In [8]:
# заполняем пропущенные значения
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())
df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))

In [9]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

В таблице не осталось пропусков.

### Замена типа данных

Для удобства расчетов предлагаю привести значения к int64. Для этого используем метод `astype`

In [10]:
columns_to_replace = ['days_employed', 'total_income']
for row in columns_to_replace:
    df[row] = df[row].astype('int')

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [12]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Благодаря замене вещественного типа данных на целочисленный таблицу стало удобнее читать.

### Обработка дубликатов

При первом знакомстве с таблицей бросились в глаза неявные дубликаты в столбце `education`. Поэтому необходимо привести все значения этого столбца к одному регистру - нижнему 

In [13]:
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Такие дубликаты возникают в результате неправильного заполнения данных.

In [14]:
df['education'] = df['education'].str.lower()

In [15]:
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Теперь у нас вместо 15 видов образования всего лишь 5 разновидностей.

Посчитаем явные дубликаты в таблице:

In [16]:
# подсчёт явных дубликатов
df.duplicated().sum()

71

In [17]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop = True)

In [18]:
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Мы убрали явные и неявные дубликаты, то есть данные стали чище. А это означает, что мы сможем сделать более точные выводы.

### Лемматизация

Для категоризации данных необходимо выделить леммы в значениях столбца с целями получения кредита.

In [21]:
def lemma_purpose(purpose):
    lemma = ' ' .join(m.lemmatize(purpose))
    return lemma

df['purpose_lemmatized'] = df['purpose'].apply(lemma_purpose)

In [22]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemmatized
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье \n
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье \n
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование \n
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба \n
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья \n


In [23]:
df['purpose_lemmatized'].unique()

array(['покупка   жилье \n', 'приобретение   автомобиль \n',
       'дополнительный   образование \n', 'сыграть   свадьба \n',
       'операция   с   жилье \n', 'образование \n',
       'на   проведение   свадьба \n', 'покупка   жилье   для   семья \n',
       'покупка   недвижимость \n',
       'покупка   коммерческий   недвижимость \n',
       'покупка   жилой   недвижимость \n',
       'строительство   собственный   недвижимость \n', 'недвижимость \n',
       'строительство   недвижимость \n',
       'на   покупка   подержать   автомобиль \n',
       'на   покупка   свой   автомобиль \n',
       'операция   с   коммерческий   недвижимость \n',
       'строительство   жилой   недвижимость \n', 'жилье \n',
       'операция   со   свой   недвижимость \n', 'автомобиль \n',
       'заниматься   образование \n',
       'сделка   с   подержанный   автомобиль \n',
       'получение   образование \n', 'свадьба \n',
       'получение   дополнительный   образование \n',
       'покупка   свой 

Можно выделить четыре группы значений: **автомобиль, образование, свадьба и недвижимость**

**Вывод**

Благодаря лемматизации в следующем разделе мы сможем разбить по категориям цели кредита.

### Категоризация данных

Создадим фунцию, которая будет принимать значения из столбца с лемматизированными значениями целей кредита и возвращать категорию в соседней ячейке столбца `purpose_cat`



In [24]:
def main_purpose(lemmas):
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'недвижимость' or 'жилье' in lemmas:
        return 'недвижимость'

df['main_purpose'] = df['purpose_lemmatized'].apply(main_purpose)

In [25]:
df['main_purpose'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: main_purpose, dtype: int64

Также категоризируем данные по признаку дохода. Можно выделить три группы: люди с доходом до 100 тысяч, от 100 до 200 и выше 200.

Для этого создадим фунцкию

In [26]:
def define_total_income(row):
    if row <= 100000:
        return 'низкая'
    elif 100000 < row <= 200000:
        return 'средняя'
    else:
        return 'высокая'
df['income_status'] = df['total_income'].apply(define_total_income)

In [27]:
df['income_status'].value_counts()

средняя    11925
высокая     5066
низкая      4463
Name: income_status, dtype: int64

**Вывод**

Для ответа на третий вопрос исследования (**Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**) мы осуществили категоризацию данных по признаку - уровень дохода.

Для ответа на четвертый вопрос исследования (**Как разные цели кредита влияют на его возврат в срок?**) мы осуществили категоризацию данных по признаку - цель кредита.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на данный вопрос необходимо сгруппировать данные по наличию детей и возвратом кредита в срок.

In [28]:
df.groupby(['children','debt']).agg({'debt':["count"]})

debt
               count
children debt       
0        0     13028
         1      1063
1        0      4410
         1       445
2        0      1926
         1       202
3        0       303
         1        27
4        0        37
         1         4
5        0         9

In [29]:
df.groupby('children')['debt'].mean()

children
0    0.075438
1    0.091658
2    0.094925
3    0.081818
4    0.097561
5    0.000000
Name: debt, dtype: float64

In [30]:
df.pivot_table('debt', index='children', aggfunc='mean')

,debt
children,
0,0.075438
1,0.091658
2,0.094925
3,0.081818
4,0.097561
5,0.000000


**Вывод**

Как видно, бездетные заемщики немного реже допускают просрочки по кредитам, чем люди с детьми. Кроме того, люди с четырьмя детьми немного чаще допускают просрочки, чем люди с меньшими количеством детей. Людей с пятью детьми всего лишь 9, поэтому отсутствие просрочек нельзя назвать закономерным.

Зависимость есть, но незначительная.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [31]:
df.groupby(['family_status','debt']).agg({'debt':["count"]})

debt
                            count
family_status         debt       
Не женат / не замужем 0      2536
                      1       274
в разводе             0      1110
                      1        85
вдовец / вдова        0       896
                      1        63
гражданский брак      0      3763
                      1       388
женат / замужем       0     11408
                      1       931

In [32]:
df.groupby('family_status')['debt'].mean()

family_status
Не женат / не замужем    0.097509
в разводе                0.071130
вдовец / вдова           0.065693
гражданский брак         0.093471
женат / замужем          0.075452
Name: debt, dtype: float64

**Вывод**

Зависимость есть: неженатые\незамужние, а также лица, состоящие в гражданском браке, чаще допускают просрочки по кредитам по сравнению с заемщиками, которые состояли или состоят в официальном браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [33]:
df.groupby(['income_status','debt']).agg({'debt':["count"]})

debt
                    count
income_status debt       
высокая       0      4708
              1       358
низкая        0      4109
              1       354
средняя       0     10896
              1      1029

In [34]:
df.groupby('income_status')['debt'].mean()

income_status
высокая    0.070667
низкая     0.079319
средняя    0.086289
Name: debt, dtype: float64

**Вывод**

Зависимость есть: люди со средним доходом (от 100 до 200) чаще допускаюют просрочки по кредитам.

- Как разные цели кредита влияют на его возврат в срок?

In [35]:
df.groupby(['main_purpose','debt']).agg({'debt':["count"]})

debt
                   count
main_purpose debt       
автомобиль   0      3903
             1       403
недвижимость 0     10029
             1       782
образование  0      3643
             1       370
свадьба      0      2138
             1       186

In [36]:
df.groupby('main_purpose')['debt'].mean()

main_purpose
автомобиль      0.093590
недвижимость    0.072334
образование     0.092200
свадьба         0.080034
Name: debt, dtype: float64

**Вывод**

По кредитам на автомобиль и образование чаще допускают просрочки. Заемщики более ответственно подходят к кредитам на недвижимость.

## Шаг 4. Общий вывод

 По результатам исследования, можно сказать, что семейное положение и количество детей клиента влияет на факт погашения кредита в срок.
 
 Самая надежная группа заемщиков: бездетные официально зарегистрированные семьи с высоким доходом, которые берут кредит на недвижимость.